## Loading the necessary libraries for the Notebook

In [2]:
from __future__ import print_function

import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)


2023-06-02 14:26:55.821445: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-02 14:26:55.824413: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-02 14:26:55.881785: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-02 14:26:55.882907: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 14:26:56.849224: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Instructions for updating:
non-resource variables are not supported in the long term


#### Add some convenience functions to Pandas DataFrame.

In [3]:
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

### Load The datasets to be considered

In [4]:
Ratingsdf =  pd.read_csv ("Netflix_Dataset_Rating.csv")
Moviesdf = pd.read_csv ("Netflix_Dataset_Movie.csv")
MoviesInfo_df = pd.read_csv ("imdb_processed.csv")
# Create one merged DataFrame containing all the Netflix data.
MoviesDF = Moviesdf.merge(Ratingsdf, on='Movie_ID').merge(MoviesInfo_df, on=['Name','Year'])
MoviesDF.head()

,Movie_ID,Year,Name,User_ID,Rating,kind,genre,imdb_rating,vote,country,language,cast,director,composer,writer,runtime
0,17,2005,7 Seconds,2187374,4,video movie,"['Action', 'Comedy', 'Crime', 'Thriller']",4.800,7153.000,"['United Kingdom', 'Romania', 'Switzerland', '...",['English'],"['Wesley Snipes', 'Tamzin Outhwaite', 'Deobia ...",['Simon Fellows'],"['Barry Taylor', 'Neal Acree']",['Martin Wheeler'],118.000
1,17,2005,7 Seconds,2503129,5,video movie,"['Action', 'Comedy', 'Crime', 'Thriller']",4.800,7153.000,"['United Kingdom', 'Romania', 'Switzerland', '...",['English'],"['Wesley Snipes', 'Tamzin Outhwaite', 'Deobia ...",['Simon Fellows'],"['Barry Taylor', 'Neal Acree']",['Martin Wheeler'],118.000
2,17,2005,7 Seconds,263315,3,video movie,"['Action', 'Comedy', 'Crime', 'Thriller']",4.800,7153.000,"['United Kingdom', 'Romania', 'Switzerland', '...",['English'],"['Wesley Snipes', 'Tamzin Outhwaite', 'Deobia ...",['Simon Fellows'],"['Barry Taylor', 'Neal Acree']",['Martin Wheeler'],118.000
3,17,2005,7 Seconds,608309,3,video movie,"['Action', 'Comedy', 'Crime', 'Thriller']",4.800,7153.000,"['United Kingdom', 'Romania', 'Switzerland', '...",['English'],"['Wesley Snipes', 'Tamzin Outhwaite', 'Deobia ...",['Simon Fellows'],"['Barry Taylor', 'Neal Acree']",['Martin Wheeler'],118.000
4,17,2005,7 Seconds,2336678,2,video movie,"['Action', 'Comedy', 'Crime', 'Thriller']",4.800,7153.000,"['United Kingdom', 'Romania', 'Switzerland', '...",['English'],"['Wesley Snipes', 'Tamzin Outhwaite', 'Deobia ...",['Simon Fellows'],"['Barry Taylor', 'Neal Acree']",['Martin Wheeler'],118.000


#### Remove the columns that wont be of use during the recommendation algorithm 

In [5]:
new_MoviesDF = MoviesDF.drop(columns=['kind','vote','country','cast','composer','runtime'])
new_MoviesDF.head()

,Movie_ID,Year,Name,User_ID,Rating,genre,imdb_rating,language,director,writer
0,17,2005,7 Seconds,2187374,4,"['Action', 'Comedy', 'Crime', 'Thriller']",4.800,['English'],['Simon Fellows'],['Martin Wheeler']
1,17,2005,7 Seconds,2503129,5,"['Action', 'Comedy', 'Crime', 'Thriller']",4.800,['English'],['Simon Fellows'],['Martin Wheeler']
2,17,2005,7 Seconds,263315,3,"['Action', 'Comedy', 'Crime', 'Thriller']",4.800,['English'],['Simon Fellows'],['Martin Wheeler']
3,17,2005,7 Seconds,608309,3,"['Action', 'Comedy', 'Crime', 'Thriller']",4.800,['English'],['Simon Fellows'],['Martin Wheeler']
4,17,2005,7 Seconds,2336678,2,"['Action', 'Comedy', 'Crime', 'Thriller']",4.800,['English'],['Simon Fellows'],['Martin Wheeler']


### Matrix Factorization via Singular Value Decomposition

In [6]:
##Format of the ratings matrix to be one row per user and one column per movie.
Ratings_df = Ratingsdf.pivot(index = 'User_ID', columns ='Movie_ID', values = 'Rating').fillna(0)
Ratings_df.head()

Movie_ID,3,8,16,17,18,26,28,30,32,33,...,4472,4474,4478,4479,4485,4488,4490,4492,4493,4496
User_ID,,,,,,,,,,,,,,,,,,,,,
6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,0.000,...,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
7,0.000,5.000,0.000,0.000,0.000,0.000,4.000,5.000,0.000,0.000,...,3.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000
79,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,0.000,...,4.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000
97,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
134,0.000,0.000,0.000,0.000,0.000,0.000,5.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [10]:
##  de-mean the data (normalize by each users mean) and convert it from a dataframe to a numpy array.
R = Ratings_df.values
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [14]:
## Scipy and Numpy both have functions to do the singular value decomposition.
#I’m going to use the Scipy function svds because it let’s me choose how many latent factors 
#I want to use to approximate the original ratings matrix (instead of having to truncate it after).
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(sigma)

In [16]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = Ratings_df.columns)

In [20]:
def recommend_movies(predictions_df, userID, Moviesdf, Ratingsdf, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = new_MoviesDF[new_MoviesDF.User_ID == (userID)]

    print ('User {0} has already rated {1} movies.'.format(userID, new_MoviesDF.shape[0]))
    print ('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (new_MoviesDF[~new_MoviesDF['MovieID'].isin(new_MoviesDF['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'MovieID',
               right_on = 'MovieID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

already_rated, predictions = recommend_movies(preds_df,2187374, Moviesdf, Ratingsdf, 10)

IndexError: single positional indexer is out-of-bounds